In [1]:
import logging
import os
from logger import setup_logger

# Initialize logger for notebook
logger = setup_logger(name="NotebookLogger", notebook=True, level=logging.DEBUG)
logger.info("Notebook execution started.")

# Ensure all subprocess logs are visible in the notebook
logger.setLevel(logging.DEBUG)
if not any(isinstance(h, logging.StreamHandler) for h in logger.handlers):
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(name)s: %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)



Output()

INFO - Notebook execution started.


In [2]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuild, JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from metrics import SentenceTransformerStrategy
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA

# Suppress specific warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# suppress HuggingFace‐tokenizers “fork” warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

logger.info("Setting up LLM and JobApplicationBuilder.")

USE_SCORES = True

llm = ChatGPT(
    api_key=OPENAI_KEY,
    model="gpt-4.1",
    system_prompt=RESUME_WRITER_PERSONA,
    max_output_tokens=None,
    temperature=0.7
)

build = JobApplicationBuild(
    resume_tex_template_path="less_basic_template.tex",
    resume_cls_path="less_basic_template.cls",
    job_content_path="input_data/Avicado data infra and ai lead.pdf",
    user_details_content_path=r"input_data/full_resume_contents_20250913.pdf"
)

# add a scoring strategy if USE_SCORES is True
if USE_SCORES:
    build.scoring_strategy = SentenceTransformerStrategy()

generator = JobApplicationBuilder(
    llm=llm
)

try:
    build = generator.extract_job_content(build)
    logger.info("Job content extracted successfully.")

    build = generator.user_data_extraction(build)
    logger.info("User data extracted successfully.")

    build = generator.generate_resume_json(build)
    logger.info("Resume JSON generated successfully.")

    build = generator.validate_resume_json(build, print_viz_changes=True)
    logger.info("Resume JSON validated successfully.")

    build = generator.resume_json_to_resume_tex(build)
    logger.info("Resume TeX file generated successfully.")

    custom_letter_instructions = input("Enter any custom application instructions for the cover letter: ")
    build = generator.generate_cover_letter(build, custom_letter_instructions, need_pdf=False)
    logger.info("Cover letter generated successfully.")

    resume_tex_fonts, _ = LatexToolBox.extract_tex_font_dependencies(build.resume_tex_path)
    font_statuses = LatexToolBox.check_fonts_installed(resume_tex_fonts)
    if not all([v for k, v in font_statuses.items()]):
        for k, v in font_statuses.items():
            if not v:
                logger.warning(f"{k} not installed")

    generator.cleanup_files(build)
    logger.info("Temporary files cleaned up successfully.")

except Exception as e:
    logger.error(f"An error occurred: {e}", exc_info=True)

INFO - Setting up LLM and JobApplicationBuilder.
INFO - Extracting job content from the provided file.
INFO - Job Details JSON generated at: output_files/AvicadoConstructionTechnologyServicesLlc/DataInfrastruct_20250913162610/JD.json
INFO - Job content extracted successfully.
INFO - Extracting user data from the provided resume file.
INFO - User data extracted successfully.
INFO - User data extracted successfully.
INFO - Generating resume JSON from job details and user data.
INFO - Processing Resume's EXPERIENCES Section...
INFO - Processing Resume's PROJECTS Section...
INFO - Processing Resume's SKILLS Section...
INFO - Processing Resume's EDUCATIONS Section...
INFO - Processing Resume's CERTIFICATIONS Section...
INFO - Processing Resume's ACHIEVEMENTS Section...
INFO - Resume JSON saved at: output_files/AvicadoConstructionTechnologyServicesLlc/DataInfrastruct_20250913162610/resume.json
INFO - Resume JSON generated successfully.
INFO - Validation diff printed to terminal.
INFO - Resum

INFO - Converting resume JSON to LaTeX format.
INFO - Resume LaTeX file saved at: output_files/AvicadoConstructionTechnologyServicesLlc/DataInfrastruct_20250913162610/resume.tex
INFO - Resume TeX file generated successfully.
INFO - Generating cover letter.
INFO - Cover letter generated at: output_files/AvicadoConstructionTechnologyServicesLlc/DataInfrastruct_20250913162610/cover_letter.txt
INFO - Cover letter generated successfully.
Matplotlib is building the font cache; this may take a moment.
INFO - Cleaning up temporary files.
INFO - Temporary files cleaned up successfully.
INFO - Temporary files cleaned up successfully.


Edit the resume latex and cover letter text before running the next cell.

In [9]:
success = LatexToolBox.compile_resume_latex_to_pdf(
    tex_filepath=build.resume_tex_path, 
    cls_filepath=build.resume_cls_path,
    output_destination_path=build.get_job_doc_path()
)
if success:
    # Remove auxiliary files using resume base name (without extension)
    base_name = os.path.splitext(os.path.basename(build.resume_tex_path))[0]
    LatexToolBox.cleanup_latex_files(build.get_job_doc_path(), base_name)
    logger.info(f"Resume PDF is saved at {build.resume_tex_path.replace('.tex','.pdf')}")
else:
    logger.error("LaTeX compilation failed.")

cover_letter_pdf_path = build.cover_letter_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(build.cover_letter_path, cover_letter_pdf_path)
logger.info(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

generator.cleanup_files(build)
logger.info("Temporary files cleaned up successfully.")

AttributeError: 'NoneType' object has no attribute 'title'

In [ ]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

Edit the resume latex and cover letter text before running the next cell.

In [ ]:
LatexToolBox.compile_resume_latex_to_pdf(tex_filepath=resume_tex_path)
logger.info(f"Resume PDF is saved at {resume_tex_path.replace('.tex', '.pdf')}")

cover_letter_pdf_path = cover_letter_txt_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(cover_letter, cover_letter_pdf_path)
logger.info(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

In [ ]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

In [10]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuilder
from application_generator import JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

p = r"output_files/UcSanDiegoSanDiegoSupercomputerCenterSdsc/DataSystemsAnal_20250829185019/resume.tex"
logger.info(f"Checking existence of file: {p}")
logger.info(f"File exists: {os.path.exists(p)}")

LatexToolBox.compile_resume_latex_to_pdf(tex_filepath=p,
                                  cls_filepath="templates/less_basic_template.cls",
                                  output_destination_path="output_files/UcSanDiegoSanDiegoSupercomputerCenterSdsc/DataSystemsAnal_20250829185019")

INFO - Checking existence of file: output_files/UcSanDiegoSanDiegoSupercomputerCenterSdsc/DataSystemsAnal_20250829185019/resume.tex
INFO - File exists: True


True

In [ ]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuild, JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

success = LatexToolBox.compile_resume_latex_to_pdf(
    tex_filepath='output_files/Ascendion/PythonEngineerI_20250507212335/resume.tex',
    cls_filepath='templates/less_basic_template.cls',
    output_destination_path='output_files/Ascendion/PythonEngineerI_20250507212335')
if success:
    logger.info("LaTeX compilation succeeded.")
else:
    logger.error("LaTeX compilation failed.")

In [11]:
import os
from utils import LatexToolBox
from logger import setup_logger

# Initialize logger if not already done in the notebook
if 'logger' not in globals():
    logger = setup_logger(name="NotebookCompileLogger", notebook=True)
    logger.info("Logger initialized for master.tex compilation.")

# Define file paths
templates_dir = "templates"
master_tex_path = os.path.join(templates_dir, "master.tex")
resume_cls_path = os.path.join(templates_dir, "resume.cls") # master.tex uses resume.cls
output_dir = templates_dir # Output PDF in the same directory

logger.info(f"Attempting to compile {master_tex_path} to PDF in {output_dir}.")
logger.info(f"Using CLS file: {resume_cls_path}")

# Ensure the cls file specified in master.tex is resume.cls
# The compile_latex_to_pdf function handles copying the cls file with the correct name.

success = LatexToolBox.compile_resume_latex_to_pdf(
    tex_filepath=master_tex_path,
    cls_filepath=resume_cls_path, # Provide the actual path to resume.cls
    output_destination_path=output_dir
)

if success:
    pdf_output_path = os.path.join(output_dir, "master.pdf")
    logger.info(f"LaTeX compilation of {master_tex_path} succeeded. PDF saved at {pdf_output_path}")
    # Clean up auxiliary files
    base_name = os.path.splitext(os.path.basename(master_tex_path))[0]
    LatexToolBox.cleanup_latex_files(output_dir, base_name)
    logger.info(f"Auxiliary files for {base_name} cleaned up from {output_dir}.")
else:
    logger.error(f"LaTeX compilation of {master_tex_path} failed.")


INFO - Attempting to compile templates/master.tex to PDF in templates.
INFO - Using CLS file: templates/resume.cls
INFO - LaTeX compilation of templates/master.tex succeeded. PDF saved at templates/master.pdf
INFO - Auxiliary files for master cleaned up from templates.
